<a href="https://colab.research.google.com/github/nxxk23/PROJECT/blob/main/Re_CountSlopes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
hemo = pd.read_csv('/content/drive/MyDrive/Thesis Project Kidney/Hemodialysis.csv', encoding='latin1')
hemo.shape

(149987, 5)

In [3]:
import pandas as pd
lab = pd.read_csv('/content/drive/MyDrive/Thesis Project Kidney/Lab.csv')
lab.shape

(21641523, 11)

In [4]:
# Create a list from hemo['PatientUID']
patient_uid_list = hemo['PatientUID'].unique().tolist()

In [5]:
patient_uid_list1 = hemo['PatientUID'].tolist() #incorrect

In [6]:
# Remove patient UIDs from lab['PatientUID'] that are in the list
lab = lab[~lab['PatientUID'].isin(patient_uid_list)]
lab.shape

(17728574, 11)

In [7]:
lab1 = lab[~lab['PatientUID'].isin(patient_uid_list1)]
lab1['PatientUID'].nunique() #incorrect

20967

In [8]:
lab = lab[~lab['PatientUID'].isin(patient_uid_list)]
lab['PatientUID'].nunique()

20967

In [9]:
lab.isnull().any()

LabNo              False
PatientUID         False
PatientVisitUID    False
Location            True
RequestItemName    False
ResultItemName      True
CWhen              False
ResultDttm          True
ResultValue         True
Unit                True
ReferenceRange      True
dtype: bool

##**🗂️ Data eGFR of all Patient**

In [10]:
# Define the values to match
result_item_names = ['eGFR(CKD-EPI) age >=18']

# Select rows where 'ResultItemName' matches the specified values
eGFR = lab[lab['ResultItemName'].isin(result_item_names)]
eGFR
labeGFR = eGFR[['LabNo','PatientUID', 'PatientVisitUID', 'ResultItemName', 'ResultValue', 'CWhen']]
labeGFR

,LabNo,PatientUID,PatientVisitUID,ResultItemName,ResultValue,CWhen
248,1,2299,3386255,eGFR(CKD-EPI) age >=18,71.7,2559-08-15 09:51:54.913
255,1,2299,3640924,eGFR(CKD-EPI) age >=18,66.8,2559-11-11 09:40:06.053
271,1,2299,4020970,eGFR(CKD-EPI) age >=18,67.20,2560-03-31 10:18:54.363
286,1,2299,4286612,eGFR(CKD-EPI) age >=18,69.20,2560-07-05 10:40:40.763
331,1,2299,4546471,eGFR(CKD-EPI) age >=18,63.60,2560-10-03 09:19:54.710
...,...,...,...,...,...,...
21628049,159,925649,3617725,eGFR(CKD-EPI) age >=18,77.60,2560-04-17 08:54:35.740
21628297,160,925649,3617725,eGFR(CKD-EPI) age >=18,72.00,2560-04-19 08:41:45.060
21628538,161,925649,3617725,eGFR(CKD-EPI) age >=18,62.90,2560-04-21 08:47:03.167
21628775,162,925649,3617725,eGFR(CKD-EPI) age >=18,59.60,2560-04-24 08:59:53.310


## **🗒 Pivot each patientID**
to look at the value they have

In [11]:
import pandas as pd

# Assuming you have a DataFrame named 'eGFR' with the specified columns
pivot_table = pd.pivot_table(labeGFR, index=['PatientUID', 'PatientVisitUID'], values=['ResultItemName', 'ResultValue', 'CWhen'], aggfunc='first')
pivot_table

CWhen          ResultItemName  \
PatientUID PatientVisitUID                                                    
2299       3386255          2559-08-15 09:51:54.913  eGFR(CKD-EPI) age >=18   
           3640924          2559-11-11 09:40:06.053  eGFR(CKD-EPI) age >=18   
           4020970          2560-03-31 10:18:54.363  eGFR(CKD-EPI) age >=18   
           4286612          2560-07-05 10:40:40.763  eGFR(CKD-EPI) age >=18   
           4546471          2560-10-03 09:19:54.710  eGFR(CKD-EPI) age >=18   
...                                             ...                     ...   
3432056    10312896         2566-01-30 16:16:45.013  eGFR(CKD-EPI) age >=18   
           10408954         2566-02-24 21:05:12.747  eGFR(CKD-EPI) age >=18   
           10573233         2566-04-10 09:51:32.587  eGFR(CKD-EPI) age >=18   
3437526    10191122         2565-12-26 09:29:11.307  eGFR(CKD-EPI) age >=18   
           10541118         2566-03-30 18:44:40.160  eGFR(CKD-EPI) age >=18   

                           ResultValue  
PatientUID PatientVisitUID              
2299       3386255                71.7  
           3640924                66.8  
           4020970               67.20  
           4286612               69.20  
           4546471               63.60  
...                                ...  
3432056    10312896              72.86  
           10408954              68.69  
           10573233              45.08  
3437526    10191122              55.94  
           10541118              56.99  

[318686 rows x 3 columns]

In [12]:
pivot_table.isnull().any()

CWhen             False
ResultItemName    False
ResultValue       False
dtype: bool

##  **🕑 generate datetime**
    Split CWhen เพราะต้องการ generate year as คศ. Pandas มี outbound
    ให้อยู่ในระยะเวลาของ คศ. แต่ลอง convert column 'CWhen'
    เป็น datetime แล้วทำไม่ได้เลยต้อง split แล้วค่อย merge

In [13]:
import pandas as pd

df = pd.DataFrame(pivot_table)

# Splitting the 'time' column into separate columns
df[['date', 'timestamp']] = df['CWhen'].str.split(' ', expand=True)
df[['year', 'month', 'day']] = df['date'].str.split('-', expand=True)
df[['hour', 'minute', 'second']] = df['timestamp'].str.split(':', expand=True)
df[['second', 'nanosecond']] = df['second'].str.split('.', expand=True)

# Convert year column from BE to AD
df['year'] = df['year'].astype(int) - 543

# Combine columns into a datetime column
df['date'] = pd.to_datetime(df[['year', 'month', 'day',]])
# Convert timestamp to Timedelta
df['timestamp'] = pd.to_timedelta(df['timestamp'])

# Convert date to datetime
df['date'] = pd.to_datetime(df['date'])

# Merge timestamp and date columns
df['datetime'] = df['date'] + df['timestamp']

df_sorted = df.sort_values(by=['PatientUID', 'datetime'])

In [14]:
df_sorted

CWhen          ResultItemName  \
PatientUID PatientVisitUID                                                    
2299       3386255          2559-08-15 09:51:54.913  eGFR(CKD-EPI) age >=18   
           3640924          2559-11-11 09:40:06.053  eGFR(CKD-EPI) age >=18   
           4020970          2560-03-31 10:18:54.363  eGFR(CKD-EPI) age >=18   
           4286612          2560-07-05 10:40:40.763  eGFR(CKD-EPI) age >=18   
           4546471          2560-10-03 09:19:54.710  eGFR(CKD-EPI) age >=18   
...                                             ...                     ...   
3432056    10312896         2566-01-30 16:16:45.013  eGFR(CKD-EPI) age >=18   
           10408954         2566-02-24 21:05:12.747  eGFR(CKD-EPI) age >=18   
           10573233         2566-04-10 09:51:32.587  eGFR(CKD-EPI) age >=18   
3437526    10191122         2565-12-26 09:29:11.307  eGFR(CKD-EPI) age >=18   
           10541118         2566-03-30 18:44:40.160  eGFR(CKD-EPI) age >=18   

                           ResultValue       date              timestamp  \
PatientUID PatientVisitUID                                                 
2299       3386255                71.7 2016-08-15 0 days 09:51:54.913000   
           3640924                66.8 2016-11-11 0 days 09:40:06.053000   
           4020970               67.20 2017-03-31 0 days 10:18:54.363000   
           4286612               69.20 2017-07-05 0 days 10:40:40.763000   
           4546471               63.60 2017-10-03 0 days 09:19:54.710000   
...                                ...        ...                    ...   
3432056    10312896              72.86 2023-01-30 0 days 16:16:45.013000   
           10408954              68.69 2023-02-24 0 days 21:05:12.747000   
           10573233              45.08 2023-04-10 0 days 09:51:32.587000   
3437526    10191122              55.94 2022-12-26 0 days 09:29:11.307000   
           10541118              56.99 2023-03-30 0 days 18:44:40.160000   

                            year month day hour minute second nanosecond  \
PatientUID PatientVisitUID                                                 
2299       3386255          2016    08  15   09     51     54        913   
           3640924          2016    11  11   09     40     06        053   
           4020970          2017    03  31   10     18     54        363   
           4286612          2017    07  05   10     40     40        763   
           4546471          2017    10  03   09     19     54        710   
...                          ...   ...  ..  ...    ...    ...        ...   
3432056    10312896         2023    01  30   16     16     45        013   
           10408954         2023    02  24   21     05     12        747   
           10573233         2023    04  10   09     51     32        587   
3437526    10191122         2022    12  26   09     29     11        307   
           10541118         2023    03  30   18     44     40        160   

                                          datetime  
PatientUID PatientVisitUID                          
2299       3386255         2016-08-15 09:51:54.913  
           3640924         2016-11-11 09:40:06.053  
           4020970         2017-03-31 10:18:54.363  
           4286612         2017-07-05 10:40:40.763  
           4546471         2017-10-03 09:19:54.710  
...                                            ...  
3432056    10312896        2023-01-30 16:16:45.013  
           10408954        2023-02-24 21:05:12.747  
           10573233        2023-04-10 09:51:32.587  
3437526    10191122        2022-12-26 09:29:11.307  
           10541118        2023-03-30 18:44:40.160  

[318686 rows x 13 columns]

## ✂ **Cut Data**



> **Patient**
```
 ให้ทำการคำนวณสโลปเฉพาะค่าที่ time_diff ห่างกันอย่างน้อย 7 วันเป็นอย่างต่ำ ถ้าน้อยกว่านั้นไม่นำมาคำนวณหาสโลป
 ให้คำนวณ time difference แบบ lag1, lag2, ..., lag6
 แล้วถ้าเจอว่า lag1 ของ time[i] (time[i] - time[i-1]) น้อยกว่าศูนย์ก็นำออก
 แต่ time[i+1] ต้องเช็คว่า lag2 < 7 ไหม ถ้าใช่ก็นำออก ถ้าไม่ใช่ก็เก็บ
```





> **Prof. Suggest**

```
 for id in patientID:
    validVisitID = [visit0]
    visitBefore = visit0
    for visit in patientVisit[1:]:
        visitNow = visit
        if timeVisitNow - timeVisitBefore >= 7:
               validVisitID.append(visit)
               visitBefore = visit

```



In [15]:
import pandas as pd
# Calculate the time differences for lags 1 to 6 and store them in new columns
for lag in range(1, 7):
    lag_column_name = f'lag{lag}'
    df_sorted[lag_column_name] = df_sorted.groupby('PatientUID')['datetime'].diff(lag)
df_sorted

CWhen          ResultItemName  \
PatientUID PatientVisitUID                                                    
2299       3386255          2559-08-15 09:51:54.913  eGFR(CKD-EPI) age >=18   
           3640924          2559-11-11 09:40:06.053  eGFR(CKD-EPI) age >=18   
           4020970          2560-03-31 10:18:54.363  eGFR(CKD-EPI) age >=18   
           4286612          2560-07-05 10:40:40.763  eGFR(CKD-EPI) age >=18   
           4546471          2560-10-03 09:19:54.710  eGFR(CKD-EPI) age >=18   
...                                             ...                     ...   
3432056    10312896         2566-01-30 16:16:45.013  eGFR(CKD-EPI) age >=18   
           10408954         2566-02-24 21:05:12.747  eGFR(CKD-EPI) age >=18   
           10573233         2566-04-10 09:51:32.587  eGFR(CKD-EPI) age >=18   
3437526    10191122         2565-12-26 09:29:11.307  eGFR(CKD-EPI) age >=18   
           10541118         2566-03-30 18:44:40.160  eGFR(CKD-EPI) age >=18   

                           ResultValue       date              timestamp  \
PatientUID PatientVisitUID                                                 
2299       3386255                71.7 2016-08-15 0 days 09:51:54.913000   
           3640924                66.8 2016-11-11 0 days 09:40:06.053000   
           4020970               67.20 2017-03-31 0 days 10:18:54.363000   
           4286612               69.20 2017-07-05 0 days 10:40:40.763000   
           4546471               63.60 2017-10-03 0 days 09:19:54.710000   
...                                ...        ...                    ...   
3432056    10312896              72.86 2023-01-30 0 days 16:16:45.013000   
           10408954              68.69 2023-02-24 0 days 21:05:12.747000   
           10573233              45.08 2023-04-10 0 days 09:51:32.587000   
3437526    10191122              55.94 2022-12-26 0 days 09:29:11.307000   
           10541118              56.99 2023-03-30 0 days 18:44:40.160000   

                            year month day hour minute second nanosecond  \
PatientUID PatientVisitUID                                                 
2299       3386255          2016    08  15   09     51     54        913   
           3640924          2016    11  11   09     40     06        053   
           4020970          2017    03  31   10     18     54        363   
           4286612          2017    07  05   10     40     40        763   
           4546471          2017    10  03   09     19     54        710   
...                          ...   ...  ..  ...    ...    ...        ...   
3432056    10312896         2023    01  30   16     16     45        013   
           10408954         2023    02  24   21     05     12        747   
           10573233         2023    04  10   09     51     32        587   
3437526    10191122         2022    12  26   09     29     11        307   
           10541118         2023    03  30   18     44     40        160   

                                          datetime                     lag1  \
PatientUID PatientVisitUID                                                    
2299       3386255         2016-08-15 09:51:54.913                      NaT   
           3640924         2016-11-11 09:40:06.053  87 days 23:48:11.140000   
           4020970         2017-03-31 10:18:54.363 140 days 00:38:48.310000   
           4286612         2017-07-05 10:40:40.763  96 days 00:21:46.400000   
           4546471         2017-10-03 09:19:54.710  89 days 22:39:13.947000   
...                                            ...                      ...   
3432056    10312896        2023-01-30 16:16:45.013   3 days 03:54:02.013000   
           10408954        2023-02-24 21:05:12.747  25 days 04:48:27.734000   
           10573233        2023-04-10 09:51:32.587  44 days 12:46:19.840000   
3437526    10191122        2022-12-26 09:29:11.307                      NaT   
           10541118        2023-03-30 18:44:40.160  94 days 09:15:28.853000   

            

In [16]:

# Function to handle conditions for lag1 to lag6
def process_lag(x):
    if x <= pd.Timedelta(days=7):
        return pd.NaT
    else:
        return x

# Apply the 'process_lag' function to the lag columns (lag1 to lag6)
for lag in range(1, 7):
    lag_column_name = f'lag{lag}'
    df_sorted[lag_column_name] = df_sorted[lag_column_name].apply(process_lag)

# Function to determine if a row is valid or invalid based on the conditions
def is_valid(row):
    if pd.notna(row['lag1']) and pd.notna(row['lag2']):
        return 'YES'
    else:
        return 'NO'

# Apply the 'is_valid' function to create the 'valid' column
df_sorted['valid'] = df_sorted.apply(is_valid, axis=1)

# Drop unnecessary columns
df_sorted.drop(columns=['date', 'timestamp', 'year', 'month', 'day', 'hour', 'minute', 'second', 'nanosecond'], inplace=True)

# Print the final DataFrame with the 'valid' column
df_sorted


CWhen          ResultItemName  \
PatientUID PatientVisitUID                                                    
2299       3386255          2559-08-15 09:51:54.913  eGFR(CKD-EPI) age >=18   
           3640924          2559-11-11 09:40:06.053  eGFR(CKD-EPI) age >=18   
           4020970          2560-03-31 10:18:54.363  eGFR(CKD-EPI) age >=18   
           4286612          2560-07-05 10:40:40.763  eGFR(CKD-EPI) age >=18   
           4546471          2560-10-03 09:19:54.710  eGFR(CKD-EPI) age >=18   
...                                             ...                     ...   
3432056    10312896         2566-01-30 16:16:45.013  eGFR(CKD-EPI) age >=18   
           10408954         2566-02-24 21:05:12.747  eGFR(CKD-EPI) age >=18   
           10573233         2566-04-10 09:51:32.587  eGFR(CKD-EPI) age >=18   
3437526    10191122         2565-12-26 09:29:11.307  eGFR(CKD-EPI) age >=18   
           10541118         2566-03-30 18:44:40.160  eGFR(CKD-EPI) age >=18   

                           ResultValue                datetime  \
PatientUID PatientVisitUID                                       
2299       3386255                71.7 2016-08-15 09:51:54.913   
           3640924                66.8 2016-11-11 09:40:06.053   
           4020970               67.20 2017-03-31 10:18:54.363   
           4286612               69.20 2017-07-05 10:40:40.763   
           4546471               63.60 2017-10-03 09:19:54.710   
...                                ...                     ...   
3432056    10312896              72.86 2023-01-30 16:16:45.013   
           10408954              68.69 2023-02-24 21:05:12.747   
           10573233              45.08 2023-04-10 09:51:32.587   
3437526    10191122              55.94 2022-12-26 09:29:11.307   
           10541118              56.99 2023-03-30 18:44:40.160   

                                               lag1                     lag2  \
PatientUID PatientVisitUID                                                     
2299       3386255                              NaT                      NaT   
           3640924          87 days 23:48:11.140000                      NaT   
           4020970         140 days 00:38:48.310000 228 days 00:26:59.450000   
           4286612          96 days 00:21:46.400000 236 days 01:00:34.710000   
           4546471          89 days 22:39:13.947000 185 days 23:01:00.347000   
...                                             ...                      ...   
3432056    10312896                             NaT  21 days 03:44:51.793000   
           10408954         25 days 04:48:27.734000  28 days 08:42:29.747000   
           10573233         44 days 12:46:19.840000  69 days 17:34:47.574000   
3437526    10191122                             NaT                      NaT   
           10541118         94 days 09:15:28.853000                      NaT   

                                               lag3                     lag4  \
PatientUID PatientVisitUID                                                     
2299       3386255                              NaT                      NaT   
           3640924                              NaT                      NaT   
           4020970                              NaT                      NaT   
           4286612         324 days 00:48:45.850000                      NaT   
           4546471         325 days 23:39:48.657000 413 days 23:27:59.797000   
...                                             ...                      ...   
3432056    10312896         46 days 06:28:13.896000                      NaT   
           10408954         46 days 08:33:19.527000  71 days 11:16:41.630000   
           10573233         72 days 21:28:49.587000  90 days 21:19:39.367000   
3437526    10191122                             NaT                      NaT   
           10541118                             NaT                      NaT   

                                               lag5 lag6 valid  
PatientUID P

In [21]:
import pandas as pd
# Method 1 - Using loc and groupby
df_sorted[df_sorted['valid']== 'YES']

CWhen          ResultItemName  \
PatientUID PatientVisitUID                                                    
2299       4020970          2560-03-31 10:18:54.363  eGFR(CKD-EPI) age >=18   
           4286612          2560-07-05 10:40:40.763  eGFR(CKD-EPI) age >=18   
           4546471          2560-10-03 09:19:54.710  eGFR(CKD-EPI) age >=18   
           4801938          2561-01-04 09:03:28.847  eGFR(CKD-EPI) age >=18   
           5051781          2561-03-29 09:10:51.133  eGFR(CKD-EPI) age >=18   
...                                             ...                     ...   
3414078    10469047         2566-03-13 13:44:52.100  eGFR(CKD-EPI) age >=18   
3417416    10528067         2566-03-28 13:59:40.430  eGFR(CKD-EPI) age >=18   
3432056    10306813         2566-01-27 12:22:43.000  eGFR(CKD-EPI) age >=18   
           10408954         2566-02-24 21:05:12.747  eGFR(CKD-EPI) age >=18   
           10573233         2566-04-10 09:51:32.587  eGFR(CKD-EPI) age >=18   

                           ResultValue                datetime  \
PatientUID PatientVisitUID                                       
2299       4020970               67.20 2017-03-31 10:18:54.363   
           4286612               69.20 2017-07-05 10:40:40.763   
           4546471               63.60 2017-10-03 09:19:54.710   
           4801938               58.40 2018-01-04 09:03:28.847   
           5051781               62.40 2018-03-29 09:10:51.133   
...                                ...                     ...   
3414078    10469047              30.18 2023-03-13 13:44:52.100   
3417416    10528067              48.95 2023-03-28 13:59:40.430   
3432056    10306813              67.71 2023-01-27 12:22:43.000   
           10408954              68.69 2023-02-24 21:05:12.747   
           10573233              45.08 2023-04-10 09:51:32.587   

                                               lag1                     lag2  \
PatientUID PatientVisitUID                                                     
2299       4020970         140 days 00:38:48.310000 228 days 00:26:59.450000   
           4286612          96 days 00:21:46.400000 236 days 01:00:34.710000   
           4546471          89 days 22:39:13.947000 185 days 23:01:00.347000   
           4801938          92 days 23:43:34.137000 182 days 22:22:48.084000   
           5051781          84 days 00:07:22.286000 176 days 23:50:56.423000   
...                                             ...                      ...   
3414078    10469047        110 days 00:16:06.293000 133 days 01:35:14.547000   
3417416    10528067        110 days 06:35:48.647000 126 days 06:19:50.550000   
3432056    10306813         17 days 23:50:49.780000  43 days 02:34:11.883000   
           10408954         25 days 04:48:27.734000  28 days 08:42:29.747000   
           10573233         44 days 12:46:19.840000  69 days 17:34:47.574000   

                                               lag3                     lag4  \
PatientUID PatientVisitUID                                                     
2299       4020970                              NaT                      NaT   
           4286612         324 days 00:48:45.850000                      NaT   
           4546471         325 days 23:39:48.657000 413 days 23:27:59.797000   
           4801938         278 days 22:44:34.484000 418 days 23:23:22.794000   
           5051781         266 days 22:30:10.370000 362 days 22:51:56.770000   
...                                             ...                      ...   
3414078    10469047                             NaT                      NaT   
3417416    10528067                             NaT                      NaT   
3432056    10306813                             NaT                      NaT   
           10408954         46 days 08:33:19.527000  71 days 11:16:41.630000   
           10573233         72 days 21:28:49.587000  90 days 21:19:39.367000   

                                               lag5                     lag6 

In [22]:
df_sorted

CWhen          ResultItemName  \
PatientUID PatientVisitUID                                                    
2299       3386255          2559-08-15 09:51:54.913  eGFR(CKD-EPI) age >=18   
           3640924          2559-11-11 09:40:06.053  eGFR(CKD-EPI) age >=18   
           4020970          2560-03-31 10:18:54.363  eGFR(CKD-EPI) age >=18   
           4286612          2560-07-05 10:40:40.763  eGFR(CKD-EPI) age >=18   
           4546471          2560-10-03 09:19:54.710  eGFR(CKD-EPI) age >=18   
...                                             ...                     ...   
3432056    10312896         2566-01-30 16:16:45.013  eGFR(CKD-EPI) age >=18   
           10408954         2566-02-24 21:05:12.747  eGFR(CKD-EPI) age >=18   
           10573233         2566-04-10 09:51:32.587  eGFR(CKD-EPI) age >=18   
3437526    10191122         2565-12-26 09:29:11.307  eGFR(CKD-EPI) age >=18   
           10541118         2566-03-30 18:44:40.160  eGFR(CKD-EPI) age >=18   

                           ResultValue                datetime  \
PatientUID PatientVisitUID                                       
2299       3386255                71.7 2016-08-15 09:51:54.913   
           3640924                66.8 2016-11-11 09:40:06.053   
           4020970               67.20 2017-03-31 10:18:54.363   
           4286612               69.20 2017-07-05 10:40:40.763   
           4546471               63.60 2017-10-03 09:19:54.710   
...                                ...                     ...   
3432056    10312896              72.86 2023-01-30 16:16:45.013   
           10408954              68.69 2023-02-24 21:05:12.747   
           10573233              45.08 2023-04-10 09:51:32.587   
3437526    10191122              55.94 2022-12-26 09:29:11.307   
           10541118              56.99 2023-03-30 18:44:40.160   

                                               lag1                     lag2  \
PatientUID PatientVisitUID                                                     
2299       3386255                              NaT                      NaT   
           3640924          87 days 23:48:11.140000                      NaT   
           4020970         140 days 00:38:48.310000 228 days 00:26:59.450000   
           4286612          96 days 00:21:46.400000 236 days 01:00:34.710000   
           4546471          89 days 22:39:13.947000 185 days 23:01:00.347000   
...                                             ...                      ...   
3432056    10312896                             NaT  21 days 03:44:51.793000   
           10408954         25 days 04:48:27.734000  28 days 08:42:29.747000   
           10573233         44 days 12:46:19.840000  69 days 17:34:47.574000   
3437526    10191122                             NaT                      NaT   
           10541118         94 days 09:15:28.853000                      NaT   

                                               lag3                     lag4  \
PatientUID PatientVisitUID                                                     
2299       3386255                              NaT                      NaT   
           3640924                              NaT                      NaT   
           4020970                              NaT                      NaT   
           4286612         324 days 00:48:45.850000                      NaT   
           4546471         325 days 23:39:48.657000 413 days 23:27:59.797000   
...                                             ...                      ...   
3432056    10312896         46 days 06:28:13.896000                      NaT   
           10408954         46 days 08:33:19.527000  71 days 11:16:41.630000   
           10573233         72 days 21:28:49.587000  90 days 21:19:39.367000   
3437526    10191122                             NaT                      NaT   
           10541118                             NaT                      NaT   

                                               lag5 lag6 valid  
PatientUID P

In [23]:
df_sorted[(df_sorted['lag1'].isna()) & (df_sorted['valid'] == 'NO')]

CWhen          ResultItemName  \
PatientUID PatientVisitUID                                                    
2299       3386255          2559-08-15 09:51:54.913  eGFR(CKD-EPI) age >=18   
2304       3431160          2559-08-30 09:37:00.867  eGFR(CKD-EPI) age >=18   
2312       5136829          2561-04-28 14:39:32.587  eGFR(CKD-EPI) age >=18   
2354       7003273          2563-01-09 07:57:21.190  eGFR(CKD-EPI) age >=18   
           9144651          2565-02-03 09:00:22.550  eGFR(CKD-EPI) age >=18   
...                                             ...                     ...   
3426284    10105952         2565-11-29 10:47:37.917  eGFR(CKD-EPI) age >=18   
3431501    10144649         2565-12-13 13:42:48.510  eGFR(CKD-EPI) age >=18   
3432056    10154545         2565-12-15 09:48:31.117  eGFR(CKD-EPI) age >=18   
           10312896         2566-01-30 16:16:45.013  eGFR(CKD-EPI) age >=18   
3437526    10191122         2565-12-26 09:29:11.307  eGFR(CKD-EPI) age >=18   

                           ResultValue                datetime lag1  \
PatientUID PatientVisitUID                                            
2299       3386255                71.7 2016-08-15 09:51:54.913  NaT   
2304       3431160                68.9 2016-08-30 09:37:00.867  NaT   
2312       5136829               57.60 2018-04-28 14:39:32.587  NaT   
2354       7003273               56.99 2020-01-09 07:57:21.190  NaT   
           9144651               42.99 2022-02-03 09:00:22.550  NaT   
...                                ...                     ...  ...   
3426284    10105952              27.88 2022-11-29 10:47:37.917  NaT   
3431501    10144649              54.65 2022-12-13 13:42:48.510  NaT   
3432056    10154545              70.18 2022-12-15 09:48:31.117  NaT   
           10312896              72.86 2023-01-30 16:16:45.013  NaT   
3437526    10191122              55.94 2022-12-26 09:29:11.307  NaT   

                                              lag2                     lag3  \
PatientUID PatientVisitUID                                                    
2299       3386255                             NaT                      NaT   
2304       3431160                             NaT                      NaT   
2312       5136829                             NaT                      NaT   
2354       7003273                             NaT                      NaT   
           9144651         83 days 23:33:09.923000 196 days 00:17:31.173000   
...                                            ...                      ...   
3426284    10105952                            NaT                      NaT   
3431501    10144649                            NaT                      NaT   
3432056    10154545                            NaT                      NaT   
           10312896        21 days 03:44:51.793000  46 days 06:28:13.896000   
3437526    10191122                            NaT                      NaT   

                                               lag4                     lag5  \
PatientUID PatientVisitUID                                                     
2299       3386255                              NaT                      NaT   
2304       3431160                              NaT                      NaT   
2312       5136829                              NaT                      NaT   
2354       7003273                              NaT                      NaT   
           9144651         223 days 19:14:10.203000 239 days 18:02:34.610000   
...                                             ...                      ...   
3426284    10105952                             NaT                      NaT   
3431501    10144649                             NaT                      NaT   
3432056    10154545                             NaT                      NaT   
           10312896                             NaT                      NaT   
3437526    10191122                             NaT                      NaT   

                         

In [24]:
df = df_sorted[df_sorted['valid']=='YES']
df.shape

(262706, 11)

In [25]:
df = df[['ResultItemName', 'ResultValue', 'datetime', 'lag1','lag2']]
df

ResultItemName ResultValue  \
PatientUID PatientVisitUID                                       
2299       4020970          eGFR(CKD-EPI) age >=18       67.20   
           4286612          eGFR(CKD-EPI) age >=18       69.20   
           4546471          eGFR(CKD-EPI) age >=18       63.60   
           4801938          eGFR(CKD-EPI) age >=18       58.40   
           5051781          eGFR(CKD-EPI) age >=18       62.40   
...                                            ...         ...   
3414078    10469047         eGFR(CKD-EPI) age >=18       30.18   
3417416    10528067         eGFR(CKD-EPI) age >=18       48.95   
3432056    10306813         eGFR(CKD-EPI) age >=18       67.71   
           10408954         eGFR(CKD-EPI) age >=18       68.69   
           10573233         eGFR(CKD-EPI) age >=18       45.08   

                                          datetime                     lag1  \
PatientUID PatientVisitUID                                                    
2299       4020970         2017-03-31 10:18:54.363 140 days 00:38:48.310000   
           4286612         2017-07-05 10:40:40.763  96 days 00:21:46.400000   
           4546471         2017-10-03 09:19:54.710  89 days 22:39:13.947000   
           4801938         2018-01-04 09:03:28.847  92 days 23:43:34.137000   
           5051781         2018-03-29 09:10:51.133  84 days 00:07:22.286000   
...                                            ...                      ...   
3414078    10469047        2023-03-13 13:44:52.100 110 days 00:16:06.293000   
3417416    10528067        2023-03-28 13:59:40.430 110 days 06:35:48.647000   
3432056    10306813        2023-01-27 12:22:43.000  17 days 23:50:49.780000   
           10408954        2023-02-24 21:05:12.747  25 days 04:48:27.734000   
           10573233        2023-04-10 09:51:32.587  44 days 12:46:19.840000   

                                               lag2  
PatientUID PatientVisitUID                           
2299       4020970         228 days 00:26:59.450000  
           4286612         236 days 01:00:34.710000  
           4546471         185 days 23:01:00.347000  
           4801938         182 days 22:22:48.084000  
           5051781         176 days 23:50:56.423000  
...                                             ...  
3414078    10469047        133 days 01:35:14.547000  
3417416    10528067        126 days 06:19:50.550000  
3432056    10306813         43 days 02:34:11.883000  
           10408954         28 days 08:42:29.747000  
           10573233         69 days 17:34:47.574000  

[262706 rows x 5 columns]

In [26]:
import pandas as pd

# Assuming 'df' is your DataFrame

# Change the column name 'lag2' to 'time_diff'
df.rename(columns={'lag2': 'time_diff'}, inplace=True)

# Convert the 'time_diff' values to numerical format representing the number of days
df['time_diff'] = pd.to_timedelta(df['time_diff']).dt.days

# Display the updated DataFrame
df

ResultItemName ResultValue  \
PatientUID PatientVisitUID                                       
2299       4020970          eGFR(CKD-EPI) age >=18       67.20   
           4286612          eGFR(CKD-EPI) age >=18       69.20   
           4546471          eGFR(CKD-EPI) age >=18       63.60   
           4801938          eGFR(CKD-EPI) age >=18       58.40   
           5051781          eGFR(CKD-EPI) age >=18       62.40   
...                                            ...         ...   
3414078    10469047         eGFR(CKD-EPI) age >=18       30.18   
3417416    10528067         eGFR(CKD-EPI) age >=18       48.95   
3432056    10306813         eGFR(CKD-EPI) age >=18       67.71   
           10408954         eGFR(CKD-EPI) age >=18       68.69   
           10573233         eGFR(CKD-EPI) age >=18       45.08   

                                          datetime                     lag1  \
PatientUID PatientVisitUID                                                    
2299       4020970         2017-03-31 10:18:54.363 140 days 00:38:48.310000   
           4286612         2017-07-05 10:40:40.763  96 days 00:21:46.400000   
           4546471         2017-10-03 09:19:54.710  89 days 22:39:13.947000   
           4801938         2018-01-04 09:03:28.847  92 days 23:43:34.137000   
           5051781         2018-03-29 09:10:51.133  84 days 00:07:22.286000   
...                                            ...                      ...   
3414078    10469047        2023-03-13 13:44:52.100 110 days 00:16:06.293000   
3417416    10528067        2023-03-28 13:59:40.430 110 days 06:35:48.647000   
3432056    10306813        2023-01-27 12:22:43.000  17 days 23:50:49.780000   
           10408954        2023-02-24 21:05:12.747  25 days 04:48:27.734000   
           10573233        2023-04-10 09:51:32.587  44 days 12:46:19.840000   

                            time_diff  
PatientUID PatientVisitUID             
2299       4020970                228  
           4286612                236  
           4546471                185  
           4801938                182  
           5051781                176  
...                               ...  
3414078    10469047               133  
3417416    10528067               126  
3432056    10306813                43  
           10408954                28  
           10573233                69  

[262706 rows x 5 columns]

## 📈 **Calculate Slope**

In [27]:
# Convert 'ResultValue' column to numeric
df['eGFR'] = pd.to_numeric(df['ResultValue'], errors='coerce')

# Convert 'datetime' column to datetime type
df['datetime'] = pd.to_datetime(df['datetime'])

# Drop rows with missing values in 'eGFR' and 'datetime' columns
df.dropna(subset=['eGFR', 'datetime'], inplace=True)

# Sort DataFrame by 'PatientUID' and 'datetime'
df.sort_values(by=['PatientUID', 'datetime'], inplace=True)

# Calculate slopes
df['slopes'] = (df['eGFR'].diff() / df['time_diff'])

# Select desired columns
eGFR_Patient = df[['ResultItemName', 'eGFR', 'datetime', 'slopes', 'time_diff']]

In [28]:
eGFR_Patient

ResultItemName   eGFR  \
PatientUID PatientVisitUID                                  
2299       4020970          eGFR(CKD-EPI) age >=18  67.20   
           4286612          eGFR(CKD-EPI) age >=18  69.20   
           4546471          eGFR(CKD-EPI) age >=18  63.60   
           4801938          eGFR(CKD-EPI) age >=18  58.40   
           5051781          eGFR(CKD-EPI) age >=18  62.40   
...                                            ...    ...   
3414078    10469047         eGFR(CKD-EPI) age >=18  30.18   
3417416    10528067         eGFR(CKD-EPI) age >=18  48.95   
3432056    10306813         eGFR(CKD-EPI) age >=18  67.71   
           10408954         eGFR(CKD-EPI) age >=18  68.69   
           10573233         eGFR(CKD-EPI) age >=18  45.08   

                                          datetime    slopes  time_diff  
PatientUID PatientVisitUID                                               
2299       4020970         2017-03-31 10:18:54.363       NaN        228  
           4286612         2017-07-05 10:40:40.763  0.008475        236  
           4546471         2017-10-03 09:19:54.710 -0.030270        185  
           4801938         2018-01-04 09:03:28.847 -0.028571        182  
           5051781         2018-03-29 09:10:51.133  0.022727        176  
...                                            ...       ...        ...  
3414078    10469047        2023-03-13 13:44:52.100 -0.140451        133  
3417416    10528067        2023-03-28 13:59:40.430  0.148968        126  
3432056    10306813        2023-01-27 12:22:43.000  0.436279         43  
           10408954        2023-02-24 21:05:12.747  0.035000         28  
           10573233        2023-04-10 09:51:32.587 -0.342174         69  

[262698 rows x 5 columns]

# ✅ **check**


> **Check**
```
เช็คว่าแต่ละคนมีจำนวนสโลปที่คำนวณจากระยะเวลาห่างกันอย่างน้อย 3 เดือน
อย่างน้อย 2 สโลปไหม ถ้ามีไม่ครบให้ตัดคนนั้นทิ้ง
```


In [29]:
import pandas as pd

# Assuming you have a DataFrame 'eGFR_Patient' with index 'PatientUID' and columns 'datetime', 'slopes', and 'time_difference'

# Create an empty list to store the valid PatientUIDs
valid_patient_uids = []

# Group the DataFrame by index 'PatientUID'
grouped = eGFR_Patient.groupby(level='PatientUID')

# Iterate over each group
for patient_uid, group in grouped:
    # Check if the group has at least 2 rows
    if len(group) >= 3:
        # Calculate the minimum and maximum datetime in the group
        min_datetime = group['datetime'].min()
        max_datetime = group['datetime'].max()

        # Calculate the time difference in days between the minimum and maximum datetime
        time_diff = (max_datetime - min_datetime).days

        # Check if the time difference is at least 90 days (3 months)
        if time_diff >= 90:
            valid_patient_uids.append(patient_uid)

# Filter the original DataFrame based on the valid PatientUIDs
filtered_df = eGFR_Patient.loc[valid_patient_uids]

In [30]:
filtered_df

ResultItemName   eGFR  \
PatientUID PatientVisitUID                                  
2299       4020970          eGFR(CKD-EPI) age >=18  67.20   
           4286612          eGFR(CKD-EPI) age >=18  69.20   
           4546471          eGFR(CKD-EPI) age >=18  63.60   
           4801938          eGFR(CKD-EPI) age >=18  58.40   
           5051781          eGFR(CKD-EPI) age >=18  62.40   
...                                            ...    ...   
3405518    10403395         eGFR(CKD-EPI) age >=18  66.54   
           10452400         eGFR(CKD-EPI) age >=18  57.23   
3409733    10094808         eGFR(CKD-EPI) age >=18  74.29   
           10136411         eGFR(CKD-EPI) age >=18  66.42   
           10435209         eGFR(CKD-EPI) age >=18  49.13   

                                          datetime    slopes  time_diff  
PatientUID PatientVisitUID                                               
2299       4020970         2017-03-31 10:18:54.363       NaN        228  
           4286612         2017-07-05 10:40:40.763  0.008475        236  
           4546471         2017-10-03 09:19:54.710 -0.030270        185  
           4801938         2018-01-04 09:03:28.847 -0.028571        182  
           5051781         2018-03-29 09:10:51.133  0.022727        176  
...                                            ...       ...        ...  
3405518    10403395        2023-02-22 09:02:41.317  0.094018        112  
           10452400        2023-03-08 10:42:01.790 -0.095979         97  
3409733    10094808        2022-11-25 16:26:52.440  0.486667         36  
           10136411        2022-12-08 10:53:47.903 -0.271379         29  
           10435209        2023-03-02 09:24:57.310 -0.208313         83  

[258004 rows x 5 columns]

In [31]:
row_counts = filtered_df.groupby('PatientUID').size()
patient_uids_with_less_than_2_row = row_counts[row_counts < 2].index
patient_uids_with_3_row = row_counts[row_counts == 3].index
print("PatientUIDs with 3 row:")
print(patient_uids_with_3_row)

PatientUIDs with 3 row:
Int64Index([   3540,    6049,    6629,    8975,    9086,    9445,    9867,
              10474,   11161,   14825,
            ...
            3366121, 3371624, 3372632, 3373579, 3375444, 3390597, 3392744,
            3396748, 3405518, 3409733],
           dtype='int64', name='PatientUID', length=1022)


In [32]:
test = filtered_df.reset_index() # check if time_diff > 90
test

,PatientUID,PatientVisitUID,ResultItemName,eGFR,datetime,slopes,time_diff
0,2299,4020970,eGFR(CKD-EPI) age >=18,67.20,2017-03-31 10:18:54.363,NaN,228
1,2299,4286612,eGFR(CKD-EPI) age >=18,69.20,2017-07-05 10:40:40.763,0.008475,236
2,2299,4546471,eGFR(CKD-EPI) age >=18,63.60,2017-10-03 09:19:54.710,-0.030270,185
3,2299,4801938,eGFR(CKD-EPI) age >=18,58.40,2018-01-04 09:03:28.847,-0.028571,182
4,2299,5051781,eGFR(CKD-EPI) age >=18,62.40,2018-03-29 09:10:51.133,0.022727,176
...,...,...,...,...,...,...,...
257999,3405518,10403395,eGFR(CKD-EPI) age >=18,66.54,2023-02-22 09:02:41.317,0.094018,112
258000,3405518,10452400,eGFR(CKD-EPI) age >=18,57.23,2023-03-08 10:42:01.790,-0.095979,97
258001,3409733,10094808,eGFR(CKD-EPI) age >=18,74.29,2022-11-25 16:26:52.440,0.486667,36
258002,3409733,10136411,eGFR(CKD-EPI) age >=18,66.42,2022-12-08 10:53:47.903,-0.271379,29


In [33]:
test[test['PatientUID']==33070] # check if time_diff > 90

,PatientUID,PatientVisitUID,ResultItemName,eGFR,datetime,slopes,time_diff


- **Delete each first rows slope of each PatientUID**

In [34]:
# Replace 'slopes' and 'time_diff' columns of the first row for each unique 'PatientUID' in DataFrame 'filtered_df'
for id in filtered_df.index.get_level_values('PatientUID').unique():
    first_row_index = filtered_df.loc[(id, slice(None)), :].index[0]
    filtered_df.loc[first_row_index, ('slopes', 'time_diff')] = None

# Print the updated DataFrame 'filtered_df'
filtered_df

ResultItemName   eGFR  \
PatientUID PatientVisitUID                                  
2299       4020970          eGFR(CKD-EPI) age >=18  67.20   
           4286612          eGFR(CKD-EPI) age >=18  69.20   
           4546471          eGFR(CKD-EPI) age >=18  63.60   
           4801938          eGFR(CKD-EPI) age >=18  58.40   
           5051781          eGFR(CKD-EPI) age >=18  62.40   
...                                            ...    ...   
3405518    10403395         eGFR(CKD-EPI) age >=18  66.54   
           10452400         eGFR(CKD-EPI) age >=18  57.23   
3409733    10094808         eGFR(CKD-EPI) age >=18  74.29   
           10136411         eGFR(CKD-EPI) age >=18  66.42   
           10435209         eGFR(CKD-EPI) age >=18  49.13   

                                          datetime    slopes  time_diff  
PatientUID PatientVisitUID                                               
2299       4020970         2017-03-31 10:18:54.363       NaN        NaN  
           4286612         2017-07-05 10:40:40.763  0.008475      236.0  
           4546471         2017-10-03 09:19:54.710 -0.030270      185.0  
           4801938         2018-01-04 09:03:28.847 -0.028571      182.0  
           5051781         2018-03-29 09:10:51.133  0.022727      176.0  
...                                            ...       ...        ...  
3405518    10403395        2023-02-22 09:02:41.317  0.094018      112.0  
           10452400        2023-03-08 10:42:01.790 -0.095979       97.0  
3409733    10094808        2022-11-25 16:26:52.440       NaN        NaN  
           10136411        2022-12-08 10:53:47.903 -0.271379       29.0  
           10435209        2023-03-02 09:24:57.310 -0.208313       83.0  

[258004 rows x 5 columns]

In [35]:
filtered_df[filtered_df['slopes']==float('inf')]

,,ResultItemName,eGFR,datetime,slopes,time_diff
PatientUID,PatientVisitUID,,,,,


In [36]:
filtered_df[filtered_df['time_diff']==0]

,,ResultItemName,eGFR,datetime,slopes,time_diff
PatientUID,PatientVisitUID,,,,,


In [37]:
t = filtered_df.reset_index() # check if time_diff > 90
t

,PatientUID,PatientVisitUID,ResultItemName,eGFR,datetime,slopes,time_diff
0,2299,4020970,eGFR(CKD-EPI) age >=18,67.20,2017-03-31 10:18:54.363,NaN,NaN
1,2299,4286612,eGFR(CKD-EPI) age >=18,69.20,2017-07-05 10:40:40.763,0.008475,236.0
2,2299,4546471,eGFR(CKD-EPI) age >=18,63.60,2017-10-03 09:19:54.710,-0.030270,185.0
3,2299,4801938,eGFR(CKD-EPI) age >=18,58.40,2018-01-04 09:03:28.847,-0.028571,182.0
4,2299,5051781,eGFR(CKD-EPI) age >=18,62.40,2018-03-29 09:10:51.133,0.022727,176.0
...,...,...,...,...,...,...,...
257999,3405518,10403395,eGFR(CKD-EPI) age >=18,66.54,2023-02-22 09:02:41.317,0.094018,112.0
258000,3405518,10452400,eGFR(CKD-EPI) age >=18,57.23,2023-03-08 10:42:01.790,-0.095979,97.0
258001,3409733,10094808,eGFR(CKD-EPI) age >=18,74.29,2022-11-25 16:26:52.440,NaN,NaN
258002,3409733,10136411,eGFR(CKD-EPI) age >=18,66.42,2022-12-08 10:53:47.903,-0.271379,29.0


In [38]:
t[t['PatientUID']==33070] # check if time_diff > 90

,PatientUID,PatientVisitUID,ResultItemName,eGFR,datetime,slopes,time_diff


In [39]:
a = filtered_df.reset_index()

In [40]:
a

,PatientUID,PatientVisitUID,ResultItemName,eGFR,datetime,slopes,time_diff
0,2299,4020970,eGFR(CKD-EPI) age >=18,67.20,2017-03-31 10:18:54.363,NaN,NaN
1,2299,4286612,eGFR(CKD-EPI) age >=18,69.20,2017-07-05 10:40:40.763,0.008475,236.0
2,2299,4546471,eGFR(CKD-EPI) age >=18,63.60,2017-10-03 09:19:54.710,-0.030270,185.0
3,2299,4801938,eGFR(CKD-EPI) age >=18,58.40,2018-01-04 09:03:28.847,-0.028571,182.0
4,2299,5051781,eGFR(CKD-EPI) age >=18,62.40,2018-03-29 09:10:51.133,0.022727,176.0
...,...,...,...,...,...,...,...
257999,3405518,10403395,eGFR(CKD-EPI) age >=18,66.54,2023-02-22 09:02:41.317,0.094018,112.0
258000,3405518,10452400,eGFR(CKD-EPI) age >=18,57.23,2023-03-08 10:42:01.790,-0.095979,97.0
258001,3409733,10094808,eGFR(CKD-EPI) age >=18,74.29,2022-11-25 16:26:52.440,NaN,NaN
258002,3409733,10136411,eGFR(CKD-EPI) age >=18,66.42,2022-12-08 10:53:47.903,-0.271379,29.0


In [41]:
a['PatientUID'].nunique()

16309

In [42]:
a.to_excel('SlopeFiltered.xlsx', index=False)


**suggestion**
```
ควรจะเก็บแค่สโลปที่มีมากกว่า 3 rows เพราะถ้าเก็บสโลปอย่างน้อย 2 rows (2 slopes)
some patient will have only 1 slope cuz the fist slope must be deleted

```

